In [1]:
import argparse, os
import torch
import numpy as np

import os, h5py, json
from tqdm import tqdm
from PIL import Image

from skimage import transform, io, img_as_float
import torchvision.transforms as transforms
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
from Transparency.Trainers.DatasetBC import *
from Transparency.ExperimentsBC import *
from Transparency.common_code.common import *
from Transparency.Trainers.PlottingBC import generate_graphs, plot_adversarial_examples, plot_logodds_examples
from Transparency.configurations import configurations
from Transparency.Trainers.TrainerBC import Trainer, Evaluator
from Transparency.model.LR import LR

torch.backends.cudnn.enabled = False
task = "bc" 

bc


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str)
parser.add_argument("--data_dir", type=str)
parser.add_argument("--output_dir", type=str)
parser.add_argument('--encoder', type=str, choices=['cnn', 'lstm', 'average', 'all'])
parser.add_argument('--attention', type=str, choices=['tanh', 'dot', 'all'])
parser.add_argument("--gpu", type=str, default='2')
parser.add_argument("--layers", type=int, default=1)
parser.add_argument("--e_layers", type=int, default=0)

args, extras = parser.parse_known_args(args=[])
args.extras = extras

args.gpu = '0'
args.output_dir = 'outputs/mc'
args.data_dir = './'

os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

In [8]:
############################ Generate Results for Table Begin ################################## 

all_dataset_names = [ 'sst', 'yelp', '20News_sports', 'AgNews']
all_encoder_names = ['cnn', 'lstm']
all_attention_names = ['dot', 'tanh']
all_exp_names = ["rand", "ig", "att", "att*grad", "att_norm", \
    "att*grad_abs", "att*grad_only_grad", "att*grad_sign", "grad_cam", "input*grad", ]
all_exp_metric_names = ["AUCTP", "Violation", "Sufficiency", "Comprehensiveness", "RC"]
all_replace_names = ["slice_out", "zeros_mask", "att_mask"]



dataset_names = ['yelp']
encoder_names = ['lstm']
attention_names = ['tanh']
exp_names = ["rand", "att"]
# exp_names = ["ig", "rand", "att", "att*grad", "input*grad", "att_norm"]

exp_metric_names = ["Violation"]
replace_names = ["slice_out", "zeros_mask", "att_mask"]

args.batch_size = 256


total_scores_dict = {dataset_name:{} for dataset_name in dataset_names}
for dataset_name in dataset_names: # Dataset Name
    if dataset_name not in total_scores_dict:
        total_scores_dict[dataset_name] = {}
    dataset_dict = total_scores_dict[dataset_name]

    # load dataset
    args.dataset = dataset_name
    dataset = datasets[args.dataset](args)
    if args.output_dir is not None :
        dataset.output_dir = args.output_dir
    dataset.display_stats()

    # mask size setup
    dataset_len = len(dataset.test_data.y)
    minlen = len(dataset.test_data.X[0]) # X is sorted
    mask_size = 1
    group_size = 20
    if minlen > group_size:
        # in case of very long sequence, we use mask_size to reduce iteration times
        mask_size += (minlen//group_size)

    for encoder_name in encoder_names: # Model Type
        args.encoder = encoder_name
        for attention_type in attention_names: # tanh  # Attention Type
            args.attention = attention_type
            if args.attention in ['dot'] :
                e = args.encoder + '_dot'
            else:
                e = args.encoder

            config = e
            if e not in dataset_dict:
                dataset_dict[e] = {}
            
            config = configurations[config](dataset)
            latest_model_path = get_latest_model(os.path.join(config['training']['basepath'], config['training']['exp_dirname']))
            evaluator = Evaluator(dataset, latest_model_path, _type=dataset.trainer_type)
            evaluator.model.bsize = args.batch_size
            predictions, attentions, metrics = evaluator.evaluate(dataset.test_data, save_results=False, return_metrics=True)

            for exp_type in exp_names: # Explanation Type
                load_cached_exp = False
                print("#"*20, "dataset:{}, model:{}, exp_type:{} ".format(dataset_name, encoder_name+"_"+attention_type, exp_type), "#"*20)
                if exp_type not in dataset_dict[e]:
                    scores = {n:[] for n in all_exp_metric_names}
                else:
                    scores = dataset_dict[e][exp_type]
                    for exp_metric in exp_metric_names:
                        scores[exp_metric] = []
                for i, exp_metric in enumerate(exp_metric_names):
                    for delta_type in replace_names: # Replacement methods
                        print("replace_func:{}, metric:{}".format(delta_type, exp_metric))
                        torch.cuda.empty_cache()
                        outputs = evaluator.replace_and_run_experiment(dataset_name, e, dataset.test_data, exp=exp_type, \
                            delta_type=delta_type, exp_metric=exp_metric, mask_size=mask_size, plot=True, \
                            load_exp_scores=load_cached_exp, load_perturbation_scores=False)

                        for calculated_metric in outputs[-1]:
                            scores[calculated_metric].append(outputs[-1][calculated_metric])
                        
                        load_cached_exp = True

                dataset_dict[e][exp_type] = scores
                mean_scores = {n: sum(scores[n])/3 for n in scores}
                print(mean_scores)
                print(scores)


INFO - 2022-01-25 20:08:05,217 - type = rnn
INFO - 2022-01-25 20:08:05,218 - vocab_size = 38636
INFO - 2022-01-25 20:08:05,219 - embed_size = 300
INFO - 2022-01-25 20:08:05,219 - hidden_size = 128
INFO - 2022-01-25 20:08:05,220 - pre_embed = None
INFO - 2022-01-25 20:08:05,221 - e_layers = 0
INFO - 2022-01-25 20:08:05,373 - hidden_size = 256
INFO - 2022-01-25 20:08:05,374 - attention.type = tanh
INFO - 2022-01-25 20:08:05,375 - attention.type = tanh
INFO - 2022-01-25 20:08:05,376 - output_size = 5
INFO - 2022-01-25 20:08:05,377 - use_attention = True
INFO - 2022-01-25 20:08:05,377 - regularizer_attention = None
INFO - 2022-01-25 20:08:05,378 - layers = 1
INFO - 2022-01-25 20:08:05,380 - type = tanh
INFO - 2022-01-25 20:08:05,381 - hidden_size = 256


{'vocab_size': 38636, 'embed_size': 300, 'train_size': [(0, 72608), (1, 68818), (2, 73502), (3, 80623), (4, 91975)], 'test_size': [(0, 5601), (1, 5350), (2, 5674), (3, 6254), (4, 7197)], 'min_length': 7, 'max_length': 149, 'mean_length': 76.30818577334166, 'std_length': 37.82558250984736}


100%|██████████| 118/118 [00:03<00:00, 30.89it/s]


{'accuracy': 0.646329299108924}


""


...

save Violation_slice_out.pkl perturbation scores to /home/lyb/Transparency/data/bc/yelp/lstm/att_norm
num. of violators: 6049, ratio:20.11%
replace_func:zeros_mask, metric:Violation
/home/lyb/Transparency/data/bc/yelp/lstm/att_norm
load exp scores from outputs/mc_1l_relu/yelp/lstm+tanh/Sun_Jan_23_00:05:10_2022/att_norm_bz256.pkl
Violation mask size must be 1


100%|██████████| 118/118 [00:31<00:00,  3.78it/s]


save Violation_zeros_mask.pkl perturbation scores to /home/lyb/Transparency/data/bc/yelp/lstm/att_norm
num. of violators: 5291, ratio:17.59%
replace_func:att_mask, metric:Violation
/home/lyb/Transparency/data/bc/yelp/lstm/att_norm
load exp scores from outputs/mc_1l_relu/yelp/lstm+tanh/Sun_Jan_23_00:05:10_2022/att_norm_bz256.pkl
Violation mask size must be 1


100%|██████████| 118/118 [00:32<00:00,  3.65it/s]

save Violation_att_mask.pkl perturbation scores to /home/lyb/Transparency/data/bc/yelp/lstm/att_norm
num. of violators: 7436, ratio:24.72%
{'AUCTP': 0.0, 'Violation': 0.20809504389762878, 'Sufficiency': 0.0, 'Comprehensiveness': 0.0, 'RC': 0.0}
{'AUCTP': [], 'Violation': [0.20112381875514984, 0.1759209930896759, 0.2472403198480606], 'Sufficiency': [], 'Comprehensiveness': [], 'RC': []}


In [31]:
import pickle
import os

data_dir = "./data"
with open(os.path.join(data_dir, "BC_results.pkl"), 'wb') as f:
    pickle.dump(total_scores_dict, f)